In [1]:
%pylab inline
import pandas
column_names = ["sex", "length", "diameter", "height", "whole weight", 
                "shucked weight", "viscera weight", "shell weight", "rings"]
data = pandas.read_csv("abalone.data", names=column_names)
print("Number of samples: %d" % len(data))
for label in "MFI":
    data[label] = data["sex"] == label
del data["sex"]
import numpy as np
from sklearn import model_selection, tree

Y = data.rings.values
del data["rings"] # remove rings from data, so we can convert all the dataframe to a numpy 2D array.
X = data.values.astype(np.float)

interval = len(Y) / 2
ymap = {}
accum = 0
i = 0
for val in range(Y.min(), Y.max() + 1):
    delta = (Y == val).sum()
    accum += delta
    if accum > interval:
        accum = delta
        i += 1
    ymap[val] = i
mapped_y = zeros(len(Y), dtype=int)
for i, v in enumerate(Y):
    mapped_y[i] = ymap[v]
Y = mapped_y
train_x, test_x, train_y, test_y = model_selection.train_test_split(X, Y) # splits 75%/25% by default

Populating the interactive namespace from numpy and matplotlib
Number of samples: 4177


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:16: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, namedtuple, defaultdict, Sequence


In [14]:
from xgboost import XGBClassifier
model = XGBClassifier(max_depth=4, n_estimators=30)
model.fit(train_x, train_y)
model.score(test_x, test_y)

0.7138755980861244

In [17]:
16 * 30

480

In [37]:
from io import StringIO
import json

def dump_xgb(model):
    buffer = StringIO()
    dump =model.get_booster().get_dump(dump_format="json")
    return [json.loads(d) for d in dump]

In [38]:
trees = dump_xgb(model)

In [39]:
trees[0]

{'children': [{'children': [{'children': [{'children': [{'leaf': 0.128859058,
         'nodeid': 15},
        {'leaf': 0.0428571403, 'nodeid': 16}],
       'depth': 3,
       'missing': 15,
       'no': 16,
       'nodeid': 7,
       'split': 'f6',
       'split_condition': 0.0525000021,
       'yes': 15},
      {'children': [{'leaf': 0.0150579112, 'nodeid': 17},
        {'leaf': 0.130769238, 'nodeid': 18}],
       'depth': 3,
       'missing': 17,
       'no': 18,
       'nodeid': 8,
       'split': 'f4',
       'split_condition': 0.232999995,
       'yes': 17}],
     'depth': 2,
     'missing': 7,
     'no': 8,
     'nodeid': 3,
     'split': 'f1',
     'split_condition': 0.242500007,
     'yes': 7},
    {'children': [{'children': [{'leaf': 0.140477821, 'nodeid': 19},
        {'leaf': -2.55448485e-09, 'nodeid': 20}],
       'depth': 3,
       'missing': 19,
       'no': 20,
       'nodeid': 9,
       'split': 'f0',
       'split_condition': 0.452499986,
       'yes': 19},
      {'chi

In [35]:
len(trees)

90